In [1]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt 
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool
from catboost.utils import get_roc_curve
import sklearn

pd.pandas.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [124]:
train['BANKR'].value_counts()

0    730141
1      3130
Name: BANKR, dtype: int64

In [52]:
train = train.sort_values('BANKR')
train.head(5)

,ID_FIRM,P1110_B,P1110_E,P1120_B,P1120_E,P1130_B,P1130_E,P1140_B,P1140_E,P1150_B,P1150_E,P1160_B,P1160_E,P1170_B,P1170_E,P1180_B,P1180_E,P1190_B,P1190_E,P1100_B,P1100_E,P1210_B,P1210_E,P1220_B,P1220_E,P1230_B,P1230_E,P1240_B,P1240_E,P1250_B,P1250_E,P1260_B,P1260_E,P1200_B,P1200_E,P1600_B,P1600_E,P1310_B,P1310_E,P1320_B,P1320_E,P1340_B,P1340_E,P1350_B,P1350_E,P1360_B,P1360_E,P1370_B,P1370_E,P1300_B,P1300_E,P1410_B,P1410_E,P1420_B,P1420_E,P1430_B,P1430_E,P1450_B,P1450_E,P1400_B,P1400_E,P1510_B,P1510_E,P1520_B,P1520_E,P1530_B,P1530_E,P1540_B,P1540_E,P1550_B,P1550_E,P1500_B,P1500_E,P1700_B,P1700_E,P2110_B,P2110_E,P2120_B,P2120_E,P2100_B,P2100_E,P2210_B,P2210_E,P2220_B,P2220_E,P2200_B,P2200_E,P2310_B,P2310_E,P2320_B,P2320_E,P2330_B,P2330_E,P2340_B,P2340_E,P2350_B,P2350_E,P2300_B,P2300_E,P2410_B,P2410_E,P2421_B,P2421_E,P2430_B,P2430_E,P2450_B,P2450_E,P2460_B,P2460_E,P2400_B,P2400_E,BANKR
0,1,8,7,0,0,0,0,0,0,402,398.0,0,0,115,96,0,0,0,0,525,501.0,2,2,0,0,3640.0,2174.0,28672,26515,2117,1694,89,155,34520.0,30540,35045,31041,785.0,785.0,0,0,0,0,13770,13770,114,114,19949,16589.0,34618,31258,0,0,0,0,0,0,0,0.0,0,0.0,0.0,0,824,5.0,0,0,0,0,0,0,824,5,35442.0,31263.0,14702.0,14581.0,0,0.0,14702,14581,5256,5513,24917,23451,-15471,-14383.0,19246,17024,2699,2517,0,0,49,3,4884,4460.0,1639,701,78,80,345,161,0,0,0,0,0,0,1561,621,0
488670,488671,30,0,0,0,0,43,0,0,0,41.0,0,0,0,0,0,0,0,0,30,84.0,0,0,0,0,74.0,191.0,0,0,2452,1359,0,0,2526.0,1550,2556,1634,11.0,11.0,0,0,0,0,0,0,0,0,2089,1573.0,2100,1584,0,0,0,0,0,0,0,0.0,0,0.0,0.0,0,462,73.0,0,0,0,0,0,0,462,73,2562.0,1657.0,12090.0,12807.0,3603,4397.0,8487,8410,0,0,0,0,8487,8410.0,0,0,0,0,0,0,0,0,85,86.0,8402,8324,0,0,0,0,0,0,0,0,375,360,8027,7964,0
488672,488673,0,0,0,0,0,0,0,0,877,13733.0,0,0,0,0,0,659,0,0,877,14392.0,1208,5135,0,184,876.0,5232.0,979,0,170,172,104,1936,3337.0,12659,4214,27051,10.0,10.0,0,0,0,0,0,0,0,0,2484,1446.0,2494,1456,0,0,0,486,0,0,0,10873.0,0,11359.0,0.0,5458,1718,8903.0,0,0,0,0,0,0,1718,14361,4212.0,27176.0,25014.0,60669.0,16977,45369.0,8037,15300,1393,10325,4172,9021,2472,-4046.0,0,0,19,71,0,290,59,549,147,222.0,2403,-3938,639,0,0,-2,0,489,0,676,0,0,1764,-3751,0
488682,488683,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0.0,469,690,0,0,2043.0,3074.0,0,0,640,752,3427,0,6579.0,4516,6579,4516,10.0,10.0,0,0,0,0,0,0,0,0,1073,2038.0,1083,2048,0,0,0,0,0,0,0,0.0,0,0.0,3844.0,0,1730,2318.0,0,0,0,0,0,0,5574,2318,6657.0,4366.0,98017.0,75314.0,87624,66279.0,10393,9035,5809,6591,0,0,4584,2444.0,0,0,0,0,0,0,0,361,485,644.0,4099,2161,240,257,0,0,0,0,0,0,0,0,3859,1904,0
488675,488676,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,218,390,0,0,218,390.0,141,0,0,0,785.0,2125.0,0,0,376,244,0,0,1302.0,2369,1520,2759,10.0,10.0,0,0,0,0,0,0,0,0,-885,-1521.0,-875,-1511,0,2236,0,0,0,0,0,0.0,0,2236.0,0.0,0,2398,2108.0,0,0,0,0,0,0,2398,2108,1523.0,2833.0,10986.0,14685.0,10805,15715.0,181,-1030,0,0,0,0,181,-1030.0,0,0,0,0,0,0,0,0,53,56.0,128,-1086,22,0,0,0,0,0,0,0,0,0,106,-1086,0


In [88]:
small_train = train[-1*int(3130*2.5):]

In [89]:
small_train['BANKR'].value_counts()

0    4695
1    3130
Name: BANKR, dtype: int64

In [90]:
X_small = small_train.drop(["BANKR", "ID_FIRM"], axis=1).fillna(0)
y_small = small_train["BANKR"]

In [91]:
X_train, X_val, y_train, y_val = train_test_split(X_small, y_small, train_size=0.8, random_state=42)

In [118]:
model = CatBoostClassifier(scale_pos_weight=1.5)

In [119]:
model.fit(X_train,
    y_train,
    verbose=False,
    eval_set=(X_val, y_val),
    plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [120]:
y_pred = model.predict(X_val)
roc_auc_score(y_val, y_pred)

0.8466643151929062

In [121]:
y_pred = model.predict(test)

In [122]:
cnt1 = 0
cnt0 = 0
for y in y_pred:
  if y == 0: cnt0+=1
  else: cnt1+=1

print(cnt0, cnt1)

267382 46877


In [123]:
submission_df = pd.DataFrame({'ID_FIRM': test['ID_FIRM'], 'BANKR': y_pred})
submission_df.to_csv('submission1.csv', index=False)